# Environment Configuration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime

from google.colab import auth
from google.cloud import bigquery
from gspread_dataframe import set_with_dataframe

import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

import pandas_gbq
import time
from datetime import datetime
import time
import random
import json

auth.authenticate_user()
print('Authenticated')

# Project Configuration
project_id = 'foodpanda-tw-bigquery'
client = bigquery.Client(project=project_id)

In [ ]:
start_datetime = datetime.now()
print(f"Script started at: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
start_time = time.time()

# Hexagon Test Area

In [ ]:
# zone_df （Taipei east zone)

import pandas as pd
from shapely.geometry import Polygon

zone_df = client.query(f'''

SELECT
  zones.id AS zone_id,
  zones.name AS zone_name,
  zones.shape AS zone_shape
FROM `fulfillment-dwh-production.curated_data_shared.countries` AS countries
LEFT JOIN countries.cities AS cities
LEFT JOIN cities.zones AS zones
WHERE countries.country_code = "tw"
  AND zones.is_active
  AND zones.name = "Taipei east"
''').to_dataframe()

zone_df

In [ ]:
# 先在zone內每500m打一個座標點。
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.ops import transform
import pyproj


# 轉換字串為shapely Polygon對象的函數
def wkt_to_polygon(wkt_str):
    wkt_str = wkt_str.replace('POLYGON((', '').replace('))', '')
    coordinates = [tuple(map(float, coord.split())) for coord in wkt_str.split(', ')]
    return Polygon(coordinates)

# 將zone_shape欄位轉換為Polygon對象
zone_df['polygon'] = zone_df['zone_shape'].apply(wkt_to_polygon)

# 創建 GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=zone_df['polygon'])

# 設定投影（使用 WGS84）
gdf = gdf.set_crs(epsg=4326)

# 生成網格
def generate_grid(polygon, spacing=1000):
    # 轉換多邊形到 Web Mercator 投影
    project = pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform
    transformed_polygon = transform(project, polygon)

    bounds = transformed_polygon.bounds
    min_x, min_y, max_x, max_y = bounds
    x_coords = np.arange(min_x, max_x, spacing)
    y_coords = np.arange(min_y, max_y, spacing)
    grid_points = []

    for x in x_coords:
        for y in y_coords:
            point = Point(x, y)
            if transformed_polygon.contains(point):
                # 將點轉換回 WGS84
                original_point = transform(pyproj.Transformer.from_crs(3857, 4326, always_xy=True).transform, point)
                grid_points.append(original_point)

    return grid_points

# 生成網格點
grid_points = []
for polygon in gdf.geometry:
    grid_points.extend(generate_grid(polygon))

# 將網格點轉換為 GeoDataFrame
grid_gdf = gpd.GeoDataFrame(geometry=grid_points)

# 繪製多邊形和網格
fig, ax = plt.subplots(figsize=(10, 10))
gdf.boundary.plot(ax=ax, color='blue', linewidth=2, label='Polygon')  # 繪製多邊形邊界
grid_gdf.plot(ax=ax, color='green', markersize=5, label='Grid Points')  # 繪製網格點

# 設定圖例和標題
plt.legend()
plt.title('Polygon and Grid Points')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()

# # 列出所有網格的經緯度
# grid_lat_long = [(point.y, point.x) for point in grid_gdf.geometry]
# for lat, long in grid_lat_long:
#     print(f'Latitude: {lat}, Longitude: {long}')

In [ ]:
# 以台北東為例，畫出蜂巢圖。
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.ops import transform
import pyproj

# 轉換字串為shapely Polygon對象的函數
def wkt_to_polygon(wkt_str):
    wkt_str = wkt_str.replace('POLYGON((', '').replace('))', '')
    coordinates = [tuple(map(float, coord.split())) for coord in wkt_str.split(', ')]
    return Polygon(coordinates)

# 將zone_shape欄位轉換為Polygon對象
zone_df['polygon'] = zone_df['zone_shape'].apply(wkt_to_polygon)

# 創建 GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=zone_df['polygon'])

# 設定投影（使用 WGS84）
gdf = gdf.set_crs(epsg=4326)

# 函數：根據中心點生成旋轉30度的六邊形
def create_rotated_hexagon(center, size=125):
    """Create a pointy-topped hexagon centered at the specified point with a given size."""
    angle_offset = np.pi / 6  # 將角度旋轉 30 度
    angle = np.linspace(0, 2 * np.pi, 7)  # 六個邊 + 一個點來閉合六邊形
    hexagon = [
        (center.x + size * np.cos(a + angle_offset), center.y + size * np.sin(a + angle_offset))
        for a in angle
    ]
    return Polygon(hexagon)

# 生成蜂巢結構的六邊形網格
def generate_hex_grid(polygon, hex_size=250, spacing=500):  # 邊長250米，寬度約500米
    project = pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform
    transformed_polygon = transform(project, polygon)

    bounds = transformed_polygon.bounds
    min_x, min_y, max_x, max_y = bounds
    grid_points = []

    row = 0
    for y in np.arange(min_y, max_y, spacing * np.sqrt(3) / 2):
        offset = (row % 2) * (spacing / 2)  # 奇數行的水平偏移
        for x in np.arange(min_x + offset, max_x, spacing):
            point = Point(x, y)
            if transformed_polygon.contains(point):
                original_point = transform(pyproj.Transformer.from_crs(3857, 4326, always_xy=True).transform, point)
                grid_points.append(original_point)
        row += 1

    return grid_points

# 生成網格點
grid_points = []
for polygon in gdf.geometry:
    grid_points.extend(generate_hex_grid(polygon))

# 將網格點轉換為 GeoDataFrame
grid_gdf = gpd.GeoDataFrame(geometry=grid_points)

# 生成蜂巢排列的六邊形
hexagons = []
hexagon_size = 250  # 邊長250米，寬度約500米
for point in grid_gdf.geometry:
    mercator_point = transform(pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform, point)
    hexagon = create_rotated_hexagon(mercator_point, size=hexagon_size)
    hexagon = transform(pyproj.Transformer.from_crs(3857, 4326, always_xy=True).transform, hexagon)
    hexagons.append(hexagon)

# 將六邊形轉換為 GeoDataFrame
hexagons_gdf = gpd.GeoDataFrame(geometry=hexagons)

# 創建 'x' 和 'y' 列，分別表示網格點的經度和緯度
grid_gdf['x'] = grid_gdf.geometry.apply(lambda point: point.x)
grid_gdf['y'] = grid_gdf.geometry.apply(lambda point: point.y)

# 對網格點進行排序並編號，先按緯度（y）再按經度（x）
grid_gdf = grid_gdf.sort_values(by=['y', 'x'], ascending=[False, True])
grid_gdf['number_of_point'] = range(1, len(grid_gdf) + 1)

# 添加zone_id和zone_name
grid_gdf['zone_id'] = zone_df['zone_id'].iloc[0]
grid_gdf['zone_name'] = zone_df['zone_name'].iloc[0]

# 添加hexagon_shape
grid_gdf['hexagon_shape'] = hexagons_gdf['geometry']

# 創建最終的DataFrame
final_df = grid_gdf[['zone_id', 'zone_name', 'number_of_point', 'y', 'x', 'hexagon_shape']]
final_df.columns = ['zone_id', 'zone_name', 'number_of_point', 'latitude', 'longitude', 'hexagon_shape']

# 繪製多邊形、網格和六邊形
fig, ax = plt.subplots(figsize=(10, 10))
gdf.boundary.plot(ax=ax, color='blue', linewidth=2, label='Polygon')  # 繪製多邊形邊界
hexagons_gdf.boundary.plot(ax=ax, color='red', linewidth=1, label='Hexagons')  # 繪製六邊形邊界
grid_gdf.plot(ax=ax, color='green', markersize=5, label='Grid Points')  # 繪製網格點

# 在圖上顯示編號
for x, y, label in zip(grid_gdf['x'], grid_gdf['y'], grid_gdf['number_of_point']):
    ax.text(x, y, str(label), fontsize=8, ha='center', va='center', color='black')

# 設定圖例和標題
plt.legend()
plt.title('Polygon, Grid Points, and Hexagonal Grid (Honeycomb)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()

# print(final_df)

# Main Script for Hexagon

In [ ]:
# zone_df

import pandas as pd
from shapely.geometry import Polygon

zone_df = client.query(f'''

SELECT
  zones.id AS zone_id,
  zones.name AS zone_name,
  zones.shape AS zone_shape
FROM `fulfillment-dwh-production.curated_data_shared.countries` AS countries
LEFT JOIN countries.cities AS cities
LEFT JOIN cities.zones AS zones
WHERE countries.country_code = "tw"
  AND zones.is_active
  AND zones.name IN ("Fengyuan","West taichung","Dajia")

''').to_dataframe()

zone_df

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.ops import transform
import pyproj

# 轉換字串為shapely Polygon對象的函數
def wkt_to_polygon(wkt_str):
    wkt_str = wkt_str.replace('POLYGON((', '').replace('))', '').replace(')', '').replace('(', '')
    coordinates = [tuple(map(float, coord.split())) for coord in wkt_str.split(', ')]
    return Polygon(coordinates)

# 將zone_shape欄位轉換為Polygon對象
zone_df['polygon'] = zone_df['zone_shape'].apply(wkt_to_polygon)

# 創建 GeoDataFrame
gdf = gpd.GeoDataFrame(zone_df, geometry=zone_df['polygon'])

# 設定投影（使用 WGS84）
gdf = gdf.set_crs(epsg=4326)

# 函數：根據中心點生成旋轉30度的六邊形
def create_rotated_hexagon(center, size=100):  # 邊長100米，寬度約200米
    angle_offset = np.pi / 6  # 將角度旋轉 30 度
    angle = np.linspace(0, 2 * np.pi, 7)  # 六個邊 + 一個點來閉合六邊形
    hexagon = [
        (center.x + size * np.cos(a + angle_offset), center.y + size * np.sin(a + angle_offset))
        for a in angle
    ]
    return Polygon(hexagon)

# 生成蜂巢結構的六邊形網格
def generate_hex_grid(polygon, hex_size=100, spacing=200):  # 邊長100米，寬度約200米
    project = pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform
    transformed_polygon = transform(project, polygon)

    bounds = transformed_polygon.bounds
    min_x, min_y, max_x, max_y = bounds
    grid_points = []

    row = 0
    for y in np.arange(min_y, max_y, spacing * np.sqrt(3) / 2):
        offset = (row % 2) * (spacing / 2)  # 奇數行的水平偏移
        for x in np.arange(min_x + offset, max_x, spacing):
            point = Point(x, y)
            if transformed_polygon.contains(point):
                original_point = transform(pyproj.Transformer.from_crs(3857, 4326, always_xy=True).transform, point)
                grid_points.append(original_point)
        row += 1

    return grid_points

# 生成網格點
grid_points = []  # 確保 grid_points 被初始化
grid_zone_id = []  # 初始化zone_id
grid_zone_name = []  # 初始化zone_name

for idx, polygon in gdf.iterrows():
    points = generate_hex_grid(polygon['geometry'])
    grid_points.extend(points)
    grid_zone_id.extend([polygon['zone_id']] * len(points))
    grid_zone_name.extend([polygon['zone_name']] * len(points))

# 將網格點轉換為 GeoDataFrame
grid_gdf = gpd.GeoDataFrame(geometry=grid_points)

# 添加zone_id和zone_name到grid_gdf
grid_gdf['zone_id'] = grid_zone_id
grid_gdf['zone_name'] = grid_zone_name

# 生成蜂巢排列的六邊形
hexagons = []
hexagon_size = 100  # 邊長100米，寬度約200米
for point in grid_gdf.geometry:
    mercator_point = transform(pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform, point)
    hexagon = create_rotated_hexagon(mercator_point, size=hexagon_size)
    hexagon = transform(pyproj.Transformer.from_crs(3857, 4326, always_xy=True).transform, hexagon)
    hexagons.append(hexagon)

# 將六邊形轉換為 GeoDataFrame
hexagons_gdf = gpd.GeoDataFrame(geometry=hexagons)

# 創建 'x' 和 'y' 列，分別表示網格點的經度和緯度
grid_gdf['x'] = grid_gdf.geometry.apply(lambda point: point.x)
grid_gdf['y'] = grid_gdf.geometry.apply(lambda point: point.y)

# 按 zone_id、y、x 排序
grid_gdf = grid_gdf.sort_values(by=['zone_id', 'y', 'x'], ascending=[True, False, True])

# 為每個 zone_id 內部編號
grid_gdf['number_of_point'] = grid_gdf.groupby('zone_id').cumcount() + 1

# 添加hexagon_shape和hex_size
grid_gdf['hexagon_shape'] = hexagons_gdf['geometry']
grid_gdf['hex_size'] = hexagon_size

# 創建最終的DataFrame
final_df = grid_gdf[['zone_id', 'zone_name', 'number_of_point', 'y', 'x', 'hexagon_shape', 'hex_size']]
final_df.columns = ['zone_id', 'zone_name', 'number_of_point', 'latitude', 'longitude', 'hexagon_shape', 'hex_size']

# 繪製多邊形、網格和六邊形
fig, ax = plt.subplots(figsize=(10, 10))
gdf.boundary.plot(ax=ax, color='blue', linewidth=2, label='Polygon')  # 繪製多邊形邊界
hexagons_gdf.boundary.plot(ax=ax, color='red', linewidth=1, label='Hexagons')  # 繪製六邊形邊界
grid_gdf.plot(ax=ax, color='green', markersize=5, label='Grid Points')  # 繪製網格點

# 在圖上顯示編號
for x, y, label in zip(grid_gdf['x'], grid_gdf['y'], grid_gdf['number_of_point']):
    ax.text(x, y, str(label), fontsize=8, ha='center', va='center', color='black')

# 設定圖例和標題
plt.legend()
plt.title('Polygon, Grid Points, and Hexagonal Grid (Honeycomb)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()


In [ ]:
final_df

In [ ]:
pip install simplekml

In [ ]:
import simplekml
from shapely import wkt

# 創建KML檔案
kml = simplekml.Kml()

for idx, row in final_df.iterrows():
    hexagon = row['hexagon_shape']
    hexagon_name = f"{row['zone_name']}_{row['zone_id']}_{row['number_of_point']}_{row['hex_size']}m"
    description = f"zone_id: {row['zone_id']}<br>zone_name: {row['zone_name']}<br>number_of_point: {row['number_of_point']}<br>latitude: {row['latitude']}<br>longitude: {row['longitude']}<br>hex_size: {row['hex_size']}m"

    # 添加六邊形到KML
    pol = kml.newpolygon(name=hexagon_name, description=description)
    pol.outerboundaryis = [(x, y) for x, y in hexagon.exterior.coords]
    pol.style.polystyle.color = 'ff0000ff'  # 藍色
    pol.style.polystyle.fill = 1
    pol.style.polystyle.outline = 1

# 儲存為KML文件
kml.save('/content/drive/MyDrive/hexagonal_grid_300.kml')

# Upload dataframe to GCP

In [ ]:
# !pip install pandas_gbq==0.20.0

In [ ]:
import pandas as pd
from datetime import datetime
import json
import google.oauth2.credentials
import pandas_gbq

In [ ]:
with open('/content/drive/MyDrive/user_account.json') as f:
    credential = json.load(f)

credentials = google.oauth2.credentials.Credentials(
    credential['token'],
    refresh_token=credential['refresh_token'],
    token_uri=credential['token_uri'],
    client_id=credential['client_id'],
    client_secret=credential['client_secret']
)

In [ ]:
pip install shapely

In [ ]:
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon

# 假設 final_df 是你的 DataFrame，並且 hexagon_shape 欄位包含 polygon 資料
# 使用 .loc 來避免 SettingWithCopyWarning
final_df.loc[:, 'hexagon_shape'] = final_df['hexagon_shape'].apply(lambda x: str(x))

# # 檢查結果
# print(final_df['hexagon_shape'])

In [ ]:
import pandas_gbq

# Set credentials and project context for pandas_gbq
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = 'database_name'

# Specify project and table
project_id = "database_name"
table_id = 'table_name'

# Upload the DataFrame to BigQuery using pandas_gbq.to_gbq
pandas_gbq.to_gbq(
    final_df,
    destination_table=table_id,
    project_id=project_id,
    credentials=credentials,
    if_exists='append'
)

print(f"Data has been uploaded to BigQuery table '{table_id}'.")


In [ ]:
end_time = time.time()
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60
print(f"Script completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
import simplekml
from shapely import wkt

# Create KML object
kml = simplekml.Kml()

# Iterate through the DataFrame rows
for idx, row in final_df.iterrows():
    # Convert the hexagon_shape (WKT) string to a Shapely polygon
    hexagon = wkt.loads(row['hexagon_shape'])

    # Create a name and description for the hexagon
    hexagon_name = f"{row['zone_name']}_{row['zone_id']}_{row['number_of_point']}_{row['hex_size']}m"
    description = f"zone_id: {row['zone_id']}<br>zone_name: {row['zone_name']}<br>number_of_point: {row['number_of_point']}<br>latitude: {row['latitude']}<br>longitude: {row['longitude']}<br>hex_size: {row['hex_size']}m"

    # Create a KML polygon for the hexagon
    pol = kml.newpolygon(name=hexagon_name, description=description)

    # Add hexagon's outer boundary coordinates to KML
    pol.outerboundaryis = [(x, y) for x, y in hexagon.exterior.coords]

    # Optional: Set the style for the polygon (e.g., blue fill color)
    pol.style.polystyle.color = 'ff0000ff'  # Blue color in KML (AABBGGRR format)
    pol.style.polystyle.fill = 1

# Save the KML file
kml.save("/content/drive/MyDrive/doc_for_colab/hexagonal_grid_100.kml")
print("KML file 'hexagons.kml' has been created.")
